In [1]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [2]:
print(corpus)

[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [3]:
#创建转换

In [4]:
from gensim import models
tfidf = models.TfidfModel(corpus) #step 1 --初始化模型

In [5]:
doc_bow = [(0,1),(1,1)]
print(tfidf[doc_bow]) #step 2 --使用模型转化向量

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [6]:
#将转换应用到整个语料库
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [7]:
lsi_model = models.LsiModel(corpus_tfidf,id2word = dictionary,num_topics=2) #初始化一个LSI转换
corpus_lsi = lsi_model[corpus_tfidf] #在原语料库上创建一个双重装饰器：bow->tfidf->fold-in-lsi

In [8]:
lsi_model.print_topics(2)

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [10]:
for doc,as_text in zip(corpus_lsi,documents):
    print(doc,as_text)

[(0, 0.06600783396090243), (1, -0.5200703306361859)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142288), (1, -0.7609563167700061)] A survey of user opinion of computer system response time
[(0, 0.08992639972446298), (1, -0.724186062675251)] The EPS user interface management system
[(0, 0.07585847652178054), (1, -0.6320551586003424)] System and human system engineering testing of EPS
[(0, 0.1015029918497996), (1, -0.5737308483002966)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378314), (1, 0.16115180214025712)] The generation of random binary unordered trees
[(0, 0.8774787673119835), (1, 0.16758906864659295)] The intersection graph of paths in trees
[(0, 0.9098624686818582), (1, 0.14086553628718873)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569278), (1, -0.05392907566389511)] Graph minors A survey


In [11]:
#模型持久化

In [14]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-',suffix='.lsi',delete=False) as tmp:
    lsi_model.save(tmp.name) 
loaded_lsi_model = models.LsiModel.load(tmp.name)
os.unlink(tmp.name)

In [15]:
#可用替换

In [16]:
#tfidf
model = models.TfidfModel(corpus,normalize=True)

In [19]:
#潜在语义索引，LSI(LSA)
model = models.LsiModel(tfidf_corpus,id2word=dictionary,num_topics=300)